# Adversarial Networks Survey - Training

The models output are set to 1000 classes, because we are using the CIFAR 10 dataset we will freeze the network except for the final layer.

## Setup

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

import copy
import time

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.Resize((70,70)),
    transforms.RandomCrop((64,64)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

batch_size = 1000

trainset = torchvision.datasets.CIFAR10('./data', download=True, train=True, transform=transform)
trainset, valset = torch.utils.data.random_split(trainset, [40000, 10000])
trainload = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=1)
valload = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=True, num_workers=1)

testset = torchvision.datasets.CIFAR10('./data', download=True, train=False, transform=transform)
testload = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=True, num_workers=1)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
dataloaders = {"train": trainload, "val": valload}
dataset_sizes = {"train": len(trainset), "val": len(valset)}

Files already downloaded and verified
Files already downloaded and verified


In [4]:
alexnet = torchvision.models.alexnet(weights=torchvision.models.AlexNet_Weights.IMAGENET1K_V1)
googlenet = torchvision.models.googlenet(weights=torchvision.models.GoogLeNet_Weights.IMAGENET1K_V1)
mobilenet = torchvision.models.mobilenet_v3_large(weights=torchvision.models.MobileNet_V3_Large_Weights.IMAGENET1K_V1)
models = [alexnet, googlenet, mobilenet]
modelNames = ["alexnet", "googlenet", "mobilenet"]

In [5]:
alexnet

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [6]:
# for param in alexnet.parameters():
#     param.requires_grad = False
num_feats = alexnet.classifier[6].in_features
alexnet.classifier[6] = nn.Linear(num_feats, 10)

In [7]:
googlenet

GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [8]:
# for param in googlenet.parameters():
#     param.requires_grad = False
num_feats = googlenet.fc.in_features
googlenet.fc = nn.Linear(num_feats, 10)

In [9]:
mobilenet

MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        )
      )
    )
    (2): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bi

In [10]:
# for param in mobilenet.parameters():
#     param.requires_grad = False
num_feat = mobilenet.classifier[3].in_features
mobilenet.classifier[3] = nn.Linear(num_feat, 10)

In [11]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25, device="cpu"):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

## Alexnet Training

In [12]:
alexnet.to(device)

criterion = nn.CrossEntropyLoss()

optimizer_alex = optim.SGD(alexnet.parameters(), lr = 0.001, momentum=0.9)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_alex, step_size=7, gamma=0.1)

In [13]:
alexnet = train_model(alexnet, criterion, optimizer_alex, exp_lr_scheduler, num_epochs=25, device=device)
# alexnet = train_model(alexnet, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=25)

Epoch 0/24
----------
train Loss: 1.4592 Acc: 0.5035
val Loss: 1.0368 Acc: 0.6365

Epoch 1/24
----------


KeyboardInterrupt: 

## GoogLeNet Training

In [15]:
googlenet.to(device)

optimizer_google = optim.SGD(googlenet.parameters(), lr = 0.001, momentum=0.9)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_google, step_size=7, gamma=0.1)

In [16]:
googlenet = train_model(googlenet, criterion, optimizer_google, exp_lr_scheduler, num_epochs=25, device=device)

Epoch 0/24
----------
train Loss: 2.2218 Acc: 0.1857
val Loss: 2.0300 Acc: 0.3175

Epoch 1/24
----------
train Loss: 1.8983 Acc: 0.3862
val Loss: 1.7135 Acc: 0.4813

Epoch 2/24
----------


KeyboardInterrupt: 

## MobileNet V3 Large Training

In [ ]:
mobilenet.to(device)

optimizer_mobile = optim.SGD(mobilenet.parameters(), lr = 0.001, momentum=0.9)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_mobile, step_size=7, gamma=0.1)

In [ ]:
mobilenet = train_model(mobilenet, criterion, optimizer_mobile, exp_lr_scheduler, num_epochs=25, device=device)

## Saving the models

In [ ]:
torch.save({
    'alexnet_state_dict': alexnet.state_dict(),
    'googlenet_state_dict': googlenet.state_dict(),
    'mobilenet_state_dict': mobilenet.state_dict(),
    'alexnet_optimizer_state_dict': optimizer_alex.state_dict(),
    'googlenet_optimizer_state_dict': optimizer_google.state_dict(),
    'mobilenet_optimizer_state_dict': optimizer_mobile.state_dict()
}, "models/models.pt")